In [1]:
import pickle
import numpy as np
import pandas as pd
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
import pandas as pd
import requests
from datetime import datetime
import re
import pymongo
import os
from pymongo import MongoClient
path = os.path.abspath(os.getcwd())

state_gov_nm = 'NH'

mongo_username= 'CEDC_Quigley'
mongo_password= 'lpP9pCcDxQ2Y0tv8'
mongo_server = 'mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
mongo_authSource = 'admin'
mongo_authMechanism = 'SCRAM-SHA-1'
mongo_database = 'legislative_documents'
mongo_people_collection = 'legiscan_people_data'
mongo_billmeta_collection = 'legiscan_bill_metadata'
mongo_vote_collection = 'legiscan_vote_data'

client = pymongo.MongoClient(mongo_server
                        ,username=mongo_username
                        ,password=mongo_password
                        ,authSource=mongo_authSource
                        ,authMechanism=mongo_authMechanism)


_BillNumber = '{}/{}/bill_numbers.list'.format(path,state_gov_nm)

_sponsor_comittee_table = '{}/{}/sponsor_comittee_table.dict'.format(path,state_gov_nm)

_BillFeatures = '{}/{}/BillFeatures.dict'.format(path,state_gov_nm)

_BillFeaturesCSV = '{}/{}/bill_features.csv'.format(path,state_gov_nm)

#parameters:

state_gov_id = 29
year=2020
year_min=2014
years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

The next cell gets the following variables from the mongo DB connection:
Bill time stamp: used to get 
    month introduction
    year introduction
    day introduction
bill number
bill ID
Title



In [2]:
database = client[mongo_database]
collection = database[mongo_billmeta_collection]

legtext_corpus = [[0,0,['dummy','entry']]]

title =[]
bill_id=[]
body=[]
month_intro=[]
year_intro=[]
day_intro=[]
state_link=[]
sponsor_id=[]
bill_number=[]
history_=[]

for bl in collection.find({'state':state_gov_nm}).batch_size(30):
    bill_id.append(str(bl['bill_id']))
    title.append(str(bl['title']))
    body.append(str(bl['body']))
    bill_time_stamp = str(bl['status_date'])
    bill_number.append(str(bl['bill_number']))
    history_.append(str(bl['history']))
    
    if bill_time_stamp == "None":
        _time= -1
        year_intro.append(0)
        month_intro.append(0)
    else:
        _time = datetime.strptime(bill_time_stamp, "%Y-%m-%d")
        year_intro.append(int(_time.strftime("%Y")))
        month_intro.append(int(_time.strftime("%m")))
        day_intro.append(int(_time.strftime('%d')))

In [3]:
#can skip this needed a dictionary for the bill numbers for the csv to be able to locally run the sentiment notebook
#bill_numbers={'billnumber': bill_number,' billid': bill_id}

#with open(_BillNumber,'wb') as f:
#    pickle.dump(bill_numbers, f)
    
#numbers = pd.DataFrame(bill_numbers)
#numbers.to_csv('bill_numbers.csv')

The next cell gathes the following:
sponsor id
sponsor name
sponsor party
the order of the sponsor
the type of the sponsor
committe id-- this is missing so why we had the committee web scraping

In [3]:
# gathering data about the sponsor 
sponsor_id=[]
sponsor_name=[]
sponsor_party=[]
sponsor_order=[]
sponsor_type_id=[]
committee_id=[]

for i, bl in enumerate(collection.find({'state':state_gov_nm}).batch_size(30)):
    sponsor_id.append([])
    sponsor_name.append([])
    sponsor_party.append([])
    sponsor_order.append([])
    sponsor_type_id.append([])
    committee_id.append([])
    for b in bl['sponsors']:
        sponsor_id[i].append(str(b['people_id']))
        sponsor_name[i].append(str(b['name']))
        sponsor_party[i].append(str(b['party']))
        sponsor_order[i].append(str(b['sponsor_order']))
        sponsor_type_id[i].append(str(b['sponsor_type_id']))
        committee_id[i].append(str(b['committee_id']))
        
sponsor_dict ={'bill_id': bill_id
               ,'sponsor_id': sponsor_id
               ,'sponsor_party': sponsor_party
               ,'sponsor_order': sponsor_order
               ,'sponsor_type_id': sponsor_type_id
              ,'sponsor_name': sponsor_name}

In [5]:
#This will onyl work for some states.
#committee  introduction data-- missing for alot of them but can be applied to other states
#committee=[]
#for bl in (collection.find({'state':state_gov_nm}).batch_size(30)):
#    committee.append(str(bl['committee']))

#doesnt work
#committee_intro=[]
#for i in range(len(committee)-1):
#    text=committee[i].split()
#    committee_=text[len(text)-1]
#    committee_=committee_[:-1] 
#    committee_intro.append(committee_)

The following cell gathers the following variables by using the sponsor_party dict:
sponsor party vec
sponsor party marjoity 


In [4]:
sponsor_party_vec=[] #average affiliation

x=0
for i in sponsor_party:
    for j in range(len(i)):
        if i[j]=='R':
            x+=2
        if i[j]=='D':
            x+=1
        else:
            x=x
    if len(i)!=0:
        x=x/ len(i)
    sponsor_party_vec.append(x)  
    x=0

The next two cells get more data:
the dates and actions
and the year it passed and month it passed

In [5]:
dates=[]
actions=[]
for i, bl in enumerate(collection.find({'state':state_gov_nm}).batch_size(30)):
    dates.append([])
    actions.append([])
    for b in bl['history']:
        dates[i].append(str(b['date']))
        actions[i].append(str(b['action']))

In [6]:
last_date=[]
intro_date=[]
year_passed=[]
month_passed=[]

x=0
for i in range(len(dates)):
    if dates[x]!=[]:
        intro_date.append((dates[x][0]))
        last_date.append((dates[x][len(dates[x])-1]))
    else:
        intro_date.append(0)
        last_date.append(0)
    x+=1

In [7]:
print(month_passed)

[]


the following cell gets primary sponsor inforamtion

In [9]:
#primary sponsor info
primary_id=[]
primary_name=[]
primary_party=[]

x=0
for j in range(len(sponsor_name)):
    if sponsor_name[x]!=[]:
        primary_name.append((sponsor_name[x][0]))
    else:
        primary_name.append("None")
    x+=1
x=0
for j in range(len(sponsor_id)):
    if sponsor_id[x]!=[]:
        primary_id.append(int(sponsor_id[x][0]))
    else:
        primary_id.append(0)
    x+=1
x=0
for j in range(len(sponsor_party)):
    if sponsor_party[x]!=[]:
        primary_party.append((sponsor_party[x][0]))
    else:
        primary_party.append("None")
    x+=1

Get the committee it was introduced to from first action, depending on state you might not need this

In [8]:
first_action=[]
for i in range(len(actions)):
    if actions[i]!=[]:
        first_action.append((actions[i][0]))
    else:
        first_action.append("None")

#instead getting committee from the first action
committee_=["None"]*len(first_action)
for i in range(len(actions)):
    text=first_action[i].split()
    for j in range(len(text)):
        if text[j]=="to" and text[j-1]=="Referred":
            committee_[i]=(text[j+1:j+7])
            
committee_intro=[]
for i in committee_:
    if "House" in i[0:len(i)]:
        y=i[0:len(i)].index("House")
        i=i[0:y]
        committee_intro.append(i)
    elif "Senate" in i[0:len(i)]:
        y=i[0:len(i)].index("Senate")
        i=i[0:y]
        committee_intro.append(i)
    else:
        i=i
        committee_intro.append(i)
#clean up committee_intro, get rid of any upper case, or special characters
temp=" "
committee_intro2=["None"]* len(committee_intro)
for i in range(len(committee_intro)):
    committee_intro2[i]=(temp.join(committee_intro[i]))
    temp=" "

com_intro=[]  
for i in range(len(committee_intro2)):
     com_intro.append(committee_intro2[i].replace(";",'').lower(). replace("&", "and"). replace(",",''))

In [9]:
print(committee_intro[:3])

[['Ways', 'and', 'Means;'], ['Executive', 'Departments', 'and', 'Administration;'], ['Election', 'Law;']]


the following cells get these variables:
number of cosponsors
number of stages

In [11]:
#number of cosponsors
num_cosponsors=[]
x=0
for j in range(len(sponsor_id)):
    num_cosponsors.append(len(sponsor_id[x]))
    x+=1

In [12]:
#number of steps/stages
num_stages=[]
x=0
for j in range(len(actions)):
    num_stages.append(len(actions[x]))
    x+=1

Since committee data is missing, we buit the follwoing datasets:
politician dataset which has the politicians in house or senate over the year 

In [13]:
roll_call_by_year = {}


for t in years:
    roll_call_by_year[t] = []

database = client[mongo_database]
collection = database[mongo_billmeta_collection]

for l in collection.find({"$and": [{'state': state_gov_nm}, {'session.year_start': {'$in': years}}]}):
    for v in l['votes']:
        roll_call_by_year[int(l["session"]["year_start"])].append(v['roll_call_id'])


In [14]:
#builds politicians_by_year -> politicians

chamber = ["H", "S"]
politicians = {}

for t in years:
    politicians[t] = {}
    for c in chamber:
        politicians[t][c] = []

database = client[mongo_database]
collections = database[mongo_vote_collection]

for t in years:
    for i in collections.find({'roll_call_id': {'$in':  roll_call_by_year[t]}}):
        for j in i['votes']:
            person = j['people_id']
            if person in politicians[t][i['chamber']]:
                continue
            else:
                politicians[t][i['chamber']].append(j['people_id'])


using the poltician data, we gather the number of terms a sponsor served

In [15]:
#the number of terms a sponsor served        

body_= ['H', 'S']
ids=[]
party=[]

while year > year_min :  
    x= len((politicians[year]['H']))
    for i in range (x):
        ids.append(politicians[year]['H'][i])
    year-=1

while year > year_min :  
    x= len((politicians[year]['S']))
    for i in range (x):
        ids.append(politicians[year]['S'][i])
    year-=1 

# num_terms is the number of terms a sponsor served
temp=[]
mul_term=[]
num_term=[]
for j in range(len(ids)):
    temp.append([i for i, value in enumerate(ids) if value ==(ids[j])] )
    
len_term=2                         
for i in range(len(temp)):
    for j in range(len(temp[i])):
        mul_term.append(ids[temp[i][j]])
    num_term.append(len(temp[i])/len_term)

mul_term2=[]
for i in range(len(mul_term)):
    if mul_term[i-1]!=mul_term[i]:
        mul_term2.append(mul_term[i])
        
num_terms=[-1]* len(primary_id) 
for j in range(len(primary_id)):
    if primary_id[j] in mul_term2:
        temp = [i for i, value in enumerate(primary_id) if value== primary_id[j]]
        for i in range(len(temp)):
            num_terms[temp[i]]=(num_term[i])  

The following cell builds the affiliation of the politicians over the years 

In [16]:
# builds people_party_table -> politicians_party

chamber = ["H", "S"]
politicians_party = {}
people_district_table = {}

for t in years:
    politicians_party[t] = {}
    people_district_table[t] = {}
    for c in chamber:
        politicians_party[t][c] = {}
        people_district_table[t][c] = {}


database = client[mongo_database]
collection = database[mongo_people_collection]

            

for t in years:
    for c in chamber:
        for l in collection.find({'people_id': {'$in': politicians[t][c]}}):
            politicians_party[t][c][l['people_id']] = l['party_id']
            people_district_table[t][c][l['people_id']] = l['district']
                


This cell gets what the majority of the house or senate was over time to then make this variable:
maj_party; 1 if they were in majority party, 0 otherwise

In [17]:

x=0
H_majority=[]
S_majority=[]
y=0

for j in years:   
    for i in politicians_party[j]["S"]:
        if politicians_party[j]["S"][i] == '1':
            x+=1
        if politicians_party[j]["S"][i] == '2':
            x+=2
        y=x/len(politicians_party[j]["S"])
    S_majority.append(y)
    x=0
    y=0
    for i in politicians_party[j]["H"]:
        if politicians_party[j]["H"][i] == '1':
            x+=1
        if politicians_party[j]["H"][i] == '2':
            x+=2
        y=x/len(politicians_party[j]["H"])
    H_majority.append(y)
    x=0
    y=0

j=0
h=0

maj_party=[-1]* len(primary_party)
for j in range(len(year_intro)):
    for h in range(len(years)):
        if year_intro[j]==years[h] and body[j]=='H':
            if H_majority[h]>1.5 and primary_party[j]=='R':
                maj_party[j]=1
            else:
                maj_party[j]=0

            if H_majority[h]==1.5: #equal party distribution, zero for everyone
                maj_party[j]=0

            if H_majority[h]<1.5 and primary_party[j]=='D':
                maj_party[j]=1
                
        if year_intro[j]==years[h] and body[j]=='S':
            if S_majority[h]>1.5 and primary_party[j]=='R':
                maj_party[j]=1
            else:
                maj_party[j]=0

            if S_majority[h]==1.5: #equal party distribution, zero for everyone
                maj_party[j]=0

            if S_majority[h]<1.5 and primary_party[j]=='D':
                maj_party[j]=1
                
#important note: here i used body as an assumption that if the primary is in the hosue the bill is introcued
#there, this is not alwasy true, but since for NH the majority R or D is the same for the same years
#this assumption is fine, we need to go back and fix this so that it is based off on chamber

The next cell builds the committee dataset for people

In [18]:

people_info_table = {}

database = client[mongo_database]
collection = database[mongo_people_collection]

chamber_ref = {1: "H", 2: "S"}
for person in collection.find({'state_id':state_gov_id}):
    people_info_table[person['people_id']] = {'party_id': int(person['party_id']), 'state_id': int(person['state_id']), 'district': person['district'], 'chamber': chamber_ref[person['role_id']]}   


The next two cells get this variable:
time from start of calender 
this link can be used to scrape all the years for all states rather than hard code:
https://www.ncsl.org/research/about-state-legislatures/2009-legislative-session-calendar.aspx


In [19]:
#####Change this cell to bring in the legislative calendar for each state in this format
##### time from start and end of time session, manually build a variable for NH, for all states refer to:
cal_year = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
start_date= ['2009-01-07', '2010-01-06', '2011-01-05', '2012-01-04', '2013-01-02', '2014-01-08', '2015-01-07', '2016-01-06', '2017-01-04', '2018-01-03', '2019-01-02']
end_date= ['2009-07-01', '2010-07-01', '2011-07-01', '2012-06-27', '2013-07-01', '2014-06-13', '2015-07-01', '2016-06-01', '2017-06-22', '2018-06-30', '2019-06-30']


start_month=[]
start_year=[]
start_day=[]

end_month=[]
end_year=[]
end_day=[]


for i in range(len(start_date)):
        start_date[i]= datetime.strptime(start_date[i], "%Y-%m-%d")
        end_date [i]= datetime.strptime(end_date[i], "%Y-%m-%d")
       
        start_year.append(int(start_date[i].strftime("%Y")))
        end_year.append(int(end_date[i].strftime("%Y")))
        
        start_month.append(int(start_date[i].strftime("%m")))
        end_month.append(int(end_date[i].strftime("%m")))
        
        start_day.append(int(start_date[i].strftime('%d')))
        end_day.append(int(start_date[i].strftime('%d')))
        

cal_dict={'year': cal_year,
            'leg session start': start_date,
            ' leg session end': end_date}


In [20]:
_month_session=[]
_day_session=[]

for i in range(len(year_intro)):
    if year_intro[i]==2009:
        _month_session.append(month_intro[i]-start_month[0])
        _day_session.append(start_day[0])
    elif year_intro[i]==2010:
        _month_session.append(month_intro[i]-start_month[1])
        _day_session.append(start_day[1])
    elif year_intro[i]==2011:
        _month_session.append(month_intro[i]-start_month[2])
        _day_session.append(start_day[2])
    elif year_intro[i]==2012:
        _month_session.append(month_intro[i]-start_month[3])
        _day_session.append(start_day[3])
    elif year_intro[i]==2013:
        _month_session.append(month_intro[i]-start_month[4])
        _day_session.append(start_day[4])
    elif year_intro[i]==2014:
        _month_session.append(month_intro[i]-start_month[5])
        _day_session.append(start_day[5])
    elif year_intro[i]==2015:
        _month_session.append(month_intro[i]-start_month[6])
        _day_session.append(start_day[6])
    elif year_intro[i]==2016:
        _month_session.append(month_intro[i]-start_month[7])
        _day_session.append(start_day[7])
    elif year_intro[i]==2017:
        _month_session.append(month_intro[i]-start_month[8])
        _day_session.append(start_day[8])
    elif year_intro[i]==2018:
        _month_session.append(month_intro[i]-start_month[9])
        _day_session.append(start_day[9])
    elif year_intro[i]==2019:
        _month_session.append(month_intro[i]-start_month[10])
        _day_session.append(start_day[10])
    else:
        _month_session.append(-20)
        _day_session.append(-20)

for i in range(len(_month_session)):
    if _month_session[i]==0:
        temp=abs((day_intro[i] - _day_session[i]))/30
        _month_session[i]=temp   

The next cell brings in the committee table to get these variables:
sponsor on committee


In [21]:
#brings in the committee data
# this needs to be ran after 040_committee_web_scraping
with open(_sponsor_comittee_table, 'rb') as f:
    committee_table=pickle.load(f)

#cleaning up the committee data
dictlist=[]
for key, value in committee_table.items():
    temp = [key,value]
    dictlist.append(temp)

name=[]
year_committee=[]
for i in range(len(dictlist)):
    name.append(dictlist[i][0])
    year_committee.append(dictlist[i][1])
    
year_com=[]
com_pos=[]
for i in year_committee:
    year_com.append(i['year'])
    com_pos.append(i['committee/positions'])

for i in range(len(name)):
    name[i]=name[i].replace("_", " ")  
    
year_comm=[]
com_pos=[]
for i in year_committee:
    year_comm.append(i['year'])
    com_pos.append(i['committee/positions'])
    
position=[]
comm_served=[]
for i in range(len(com_pos)):
    position.append([])
    comm_served.append([])
    for j in range(len(com_pos[i])):
        if "/" in com_pos[i][j]:
            position[i].append(com_pos[i][j][(com_pos[i][j].index("/")+1):])
            comm_served[i].append(com_pos[i][j][:(com_pos[i][j].index("/"))].strip())
            
year_com2=[]
for i in (range(len(year_comm))):
    year_com2.append([])
    for j in range(len(year_comm[i])):     
        if "-" in year_comm[i][j]:
            year_com2[i].append(int(year_comm[i][j][:4]))
            year_com2[i].append(int(year_comm[i][j][5:]))
        else:
            year_com2[i].append(int(year_comm[i][j]))
            
position2=[]
for j in range(len(year_com2)):
    for i in range(len(position)):
        if len(year_com2[j])!=len(position[j]):
            position[j].append(position[j])
            
primary_on_comm=[0]*len(primary_name)
pos_on_com=[0]*len(primary_name)

for i in range(len(primary_name)):
    if primary_name[i] in name:
        x= name.index(primary_name[i])
        if com_intro[i] in comm_served[x]:
            if year_intro[i] in year_com2[x]:
                primary_on_comm[i]=1 
                h= year_com2[x].index(year_intro[i])
                pos_on_com[i]= position[x][h]


In [22]:
new_features_dict ={'bill_id':  bill_id
                    ,'title': title
                    ,'house/sentate': body
                    , 'primary name': primary_name
                    , 'primary id': primary_id
                    , 'primary party': primary_party
                    , 'number of co sponsors': num_cosponsors
                    , "committee introduced": committee_intro
                    , "number of terms served": num_terms
                    , 'primary in majority party': maj_party
                    , 'primary is on the committee': primary_on_comm
                    , 'position of primary on committee': pos_on_com
                    , 'month introduced': month_intro
                    , 'year introduced': year_intro
                    #, 'month passed': month_passed
                    #, 'year passed': year_passed
                    , "months from the start of legislative calender intro": _month_session}

In [23]:
print(len(bill_id))
print(len(title))
print(len(body))
print(len(primary_name))
print(len(primary_id))
print(len(primary_party))
print(len(num_cosponsors))
print(len(committee_intro))
print(len(num_terms))
print(len(maj_party))
print(len(primary_on_comm))
print(len(pos_on_com))
print(len(month_intro))
print(len(year_intro))
#print(len(month_passed))
#print(len(year_passed))
print(len(_month_session))

10613
10613
10613
10613
10613
10613
10613
10613
10613
10613
10613
10613
10613
10613
10613


In [24]:
#creating the target dataset for overall classifiers leg_win_state.csv
bill_features = pd.DataFrame(new_features_dict)

# creating an overall dataset with defined win condition
with open(_BillFeatures,'wb') as f:
    pickle.dump(bill_features, f)

In [25]:
bill_features.to_csv(_BillFeaturesCSV)